In [2]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, VotingClassifier

from sklearn.metrics import (accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

import matplotlib.pyplot as plt
import seaborn as sns

# add src to path so we can import our data module
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(os.path.join(project_root, "src"))

from data.load_data import download_creditcard_data, load_creditcard_df

ModuleNotFoundError: No module named 'kaggle'

# Project step 2: Grid search and ensembles on credit card fraud

In this notebook we apply grid search to our baseline models and we build ensemble models.
We work on the same credit card fraud dataset as in project step 1.

In [ ]:
# we download the data (if already downloaded this does nothing important)
download_creditcard_data()
df = load_creditcard_df()

df = df.drop_duplicates().reset_index(drop=True)

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42,)

# we check class balance on train and test
print("Train class distribution:")
print(y_train.value_counts(normalize=True))

print("\nTest class distribution:")
print(y_test.value_counts(normalize=True))

In [ ]:
# we scale only Time and Amount and keep PCA components as they are
numeric_to_scale = ["Time", "Amount"]
other_features = [col for col in X.columns if col not in numeric_to_scale + ["Class"]]

preprocessor = ColumnTransformer(
    transformers=[
        ("scale_time_amount", StandardScaler(), numeric_to_scale),
        ("pass_others", "passthrough", other_features),
    ]
)

## 1. Grid search on baseline models

We tune two baseline models:
- a Decision Tree with class weights for imbalance
- a Logistic Regression with class weights

We use F1 score for the fraud class as the main objective in the grid search.

In [ ]:
# Grid search for decision tree
dt_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", DecisionTreeClassifier(class_weight="balanced", random_state=42)),
    ]
)

param_grid_dt = {
    "model__criterion": ["gini", "entropy", "log_loss"],
    "model__max_depth": [3, 5, 7, 9, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4],
}

grid_dt = GridSearchCV(
    estimator=dt_pipe,
    param_grid=param_grid_dt,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1,
)

grid_dt.fit(X_train, y_train)

print("Best Decision Tree params:")
print(grid_dt.best_params_)
print(f"Best CV F1 score: {grid_dt.best_score_:.4f}")

best_dt = grid_dt.best_estimator_

y_pred_dt = best_dt.predict(X_test)
y_proba_dt = best_dt.predict_proba(X_test)[:, 1]

print("\nDecision Tree classification report (test):")
print(classification_report(y_test, y_pred_dt, digits=4))

In [ ]:
# Grid search for Logistic Regression
lr_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        ("model", LogisticRegression(class_weight="balanced", max_iter=1000)),
    ]
)

param_grid_lr = {
    "model__C": [0.01, 0.1, 1, 10],
    "model__penalty": ["l2"],
    "model__solver": ["liblinear", "lbfgs"],
}

grid_lr = GridSearchCV(
    estimator=lr_pipe,
    param_grid=param_grid_lr,
    scoring="f1",
    cv=3,
    n_jobs=-1,
    verbose=1,
)

grid_lr.fit(X_train, y_train)

print("Best Logistic Regression params:")
print(grid_lr.best_params_)
print(f"Best CV F1 score: {grid_lr.best_score_:.4f}")

best_lr = grid_lr.best_estimator_

y_pred_lr = best_lr.predict(X_test)
y_proba_lr = best_lr.predict_proba(X_test)[:, 1]

print("\nLogistic Regression classification report (test):")
print(classification_report(y_test, y_pred_lr, digits=4))


NameError: name 'Pipeline' is not defined

## 2. Ensemble models: Bagging and Voting

We now build:
- a Bagging ensemble based on a Decision Tree
- a Voting ensemble that combines the tuned Decision Tree and the tuned Logistic Regression

We keep the same train and test sets as before.

In [1]:
bag_dt_pipe = Pipeline(
    steps=[
        ("preprocess", preprocessor),
        (
            "model",
            BaggingClassifier(
                estimator=DecisionTreeClassifier(
                    class_weight="balanced",
                    random_state=42,
                ),
                n_estimators=50,
                max_samples=0.8,
                n_jobs=-1,
                random_state=42,
            ),
        ),
    ]
)

bag_dt_pipe.fit(X_train, y_train)

y_pred_bag = bag_dt_pipe.predict(X_test)
y_proba_bag = bag_dt_pipe.predict_proba(X_test)[:, 1]

print("Bagging Decision Tree classification report (test):")
print(classification_report(y_test, y_pred_bag, digits=4))

NameError: name 'Pipeline' is not defined

In [2]:
# we build a VotingClassifier from the best pipelines
voting_clf = VotingClassifier(
    estimators=[
        ("dt", best_dt),
        ("lr", best_lr),
    ],
    voting="soft",
)

voting_clf.fit(X_train, y_train)

y_pred_vote = voting_clf.predict(X_test)
y_proba_vote = voting_clf.predict_proba(X_test)[:, 1]

print("Voting ensemble classification report (test):")
print(classification_report(y_test, y_pred_vote, digits=4))

NameError: name 'VotingClassifier' is not defined

In [3]:
def compute_metrics(y_true, y_pred, y_proba, name):
    return {
        "model": name,
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "roc_auc": roc_auc_score(y_true, y_proba),
    }

results = []

results.append(compute_metrics(y_test, y_pred_dt, y_proba_dt, "DecisionTree (tuned)"))
results.append(compute_metrics(y_test, y_pred_lr, y_proba_lr, "LogisticRegression (tuned)"))
results.append(compute_metrics(y_test, y_pred_bag, y_proba_bag, "Bagging DT"))
results.append(compute_metrics(y_test, y_pred_vote, y_proba_vote, "Voting (DT + LR)"))

results_df = pd.DataFrame(results)
results_df

NameError: name 'y_test' is not defined